In [1]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash


--2021-12-27 09:40:03--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2021-12-27 09:40:03--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-12-27 09:40:04--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:44

In [2]:
!wget http://setup.johnsnowlabs.com/colab.sh 

! cat /content/colab.sh

--2021-12-27 09:40:57--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2021-12-27 09:40:57--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-12-27 09:40:58--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:44

In [126]:
import pandas as pd
import numpy as np

# 2.1 spark related
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType, StringType,StructField,StructType
#Replace part of string with another string
from pyspark.sql.functions import regexp_replace

# 2.2 Spark-nlp related
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

In [4]:
# 2.3 Create Spark session
#     And start sparknlp

spark = sparknlp.start()

In [5]:
# 2.4 Show multiple command outputs from a cell

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [83]:
from sparknlp.pretrained import PretrainedPipeline
pipeline = PretrainedPipeline('analyze_sentiment', lang = 'en')
result =  pipeline.annotate("Financial News ")
print(result)

analyze_sentiment download started this may take some time.
Approx size to download 4.9 MB
[OK!]
{'checked': ['Financial', 'News'], 'document': ['Financial News '], 'sentiment': ['negative'], 'token': ['Financial', 'News'], 'sentence': ['Financial News']}


In [6]:
# 2.3 Mount gdrive to read data
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [84]:
# 4.0 Read data directly in spark a normal manner:

df = spark.read.csv(
                      path = '/gdrive/MyDrive/FA Projects/all-data (1).csv',
                      inferSchema=True,
                      header=True
                      )

In [98]:
df.show(30)

+--------+--------------------+
|  Review|               Tweet|
+--------+--------------------+
| neutral|According to Gran...|
| neutral|Technopolis plans...|
|negative|The international...|
|positive|With the new prod...|
|positive|According to the ...|
|positive|FINANCING OF ASPO...|
|positive|For the last quar...|
|positive|In the third quar...|
|positive|Operating profit ...|
|positive|Operating profit ...|
|positive|TeliaSonera TLSN ...|
|positive|STORA ENSO , NORS...|
|positive|A purchase agreem...|
|positive|Finnish Talentum ...|
|positive|Clothing retail c...|
|positive|Consolidated net ...|
|positive|Foundries divisio...|
|positive|HELSINKI ( AFX ) ...|
|positive|Incap Contract Ma...|
|positive|Its board of dire...|
|positive|Lifetree was foun...|
|positive|( Filippova ) A t...|
|positive|MegaFon 's subscr...|
|positive|Net income from l...|
|positive|Net sales increas...|
|positive|Net sales surged ...|
|positive|Nordea Group 's o...|
|positive|Operating profit ...|
|positiv

In [86]:
df.count() 

4846

In [87]:
df.head(5)

[Row(Review='neutral', Tweet='According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .'),
 Row(Review='neutral', Tweet='Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .'),
 Row(Review='negative', Tweet='The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .'),
 Row(Review='positive', Tweet='With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .'),
 Row(Review='positive', Tweet="According to the company 's updated strategy for the years 2009-2012 , Basware t

In [115]:
# 5.0 First define a Spark schema:
schema = StructType([ \
                     StructField("Tweet",StringType(),True), \
                     StructField("Review",StringType(),True), \
  ])
 

In [116]:
# 5.1 Next read the data:

trainDataset = spark.read  \
                 .option("quote", "\"") \
                 .option('escape', "\"") \
                 .option("multiLine", "true")  \
                 .option("schema" , schema)  \
                 .option("header", "true") \
                 .csv('/gdrive/MyDrive/FA Projects/all-data (1).csv')

In [117]:
# 5.2 The result shows complete match with pandas:
trainDataset.show()
trainDataset.count() 

+--------+--------------------+
|  Review|               Tweet|
+--------+--------------------+
| neutral|According to Gran...|
| neutral|Technopolis plans...|
|negative|The international...|
|positive|With the new prod...|
|positive|According to the ...|
|positive|FINANCING OF ASPO...|
|positive|For the last quar...|
|positive|In the third quar...|
|positive|Operating profit ...|
|positive|Operating profit ...|
|positive|TeliaSonera TLSN ...|
|positive|STORA ENSO , NORS...|
|positive|A purchase agreem...|
|positive|Finnish Talentum ...|
|positive|Clothing retail c...|
|positive|Consolidated net ...|
|positive|Foundries divisio...|
|positive|HELSINKI ( AFX ) ...|
|positive|Incap Contract Ma...|
|positive|Its board of dire...|
+--------+--------------------+
only showing top 20 rows



4846

In [118]:
# 5.3 Distribution of classes:
from pyspark.sql.functions import col

# 5.3.1
trainDataset.groupBy("Review") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|  Review|count|
+--------+-----+
| neutral| 2879|
|positive| 1363|
|negative|  604|
+--------+-----+



In [119]:
(trainingData, testData) = trainDataset.randomSplit([0.8, 0.2], seed = 100)

# 6.0.1
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 3888
Test Dataset Count: 958


In [120]:
#pipelinig for data cleaning
# 6.1
document_assembler = DocumentAssembler() \
    .setInputCol("Tweet") \
    .setOutputCol("document")

# 6.2    
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# 6.3      
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

# 6.4
stopwords_cleaner = StopWordsCleaner()\
    .setInputCols("normalized")\
    .setOutputCol("cleanTokens")\
    .setCaseSensitive(False)

# 6.5
lemma = LemmatizerModel.pretrained('lemma_antbnc') \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma")

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [121]:
# 7.0
glove_embeddings = WordEmbeddingsModel().pretrained() \
      .setInputCols(["document",'lemma'])\
      .setOutputCol("embeddings")\
      .setCaseSensitive(False)

# 7.1
embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

# 7.2
classsifierdl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("Review")\
      .setMaxEpochs(100)\
      .setEnableOutputLogs(True)
      #.setOutputLogsPath('logs')

# 7.3
clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            embeddingsSentence,
            classsifierdl])

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [122]:
# 8.0 Train 

clf_pipelineModel = clf_pipeline.fit(trainingData)

In [123]:
preds = clf_pipelineModel.transform(testData)


In [124]:
preds.select('Tweet','Review',"class.result").show(10, truncate=80)


+--------------------------------------------------------------------------------+--------+---------+
|                                                                           Tweet|  Review|   result|
+--------------------------------------------------------------------------------+--------+---------+
|( ADP News ) - Jan 22 , 2009 - Finnish mobile phones maker Nokia Oyj ( OMX : ...|negative|[neutral]|
|- Net sales for the period are expected to fall well below that of last year ...|negative|[neutral]|
|17 March 2011 - Goldman Sachs estimates that there are negative prospects for...|negative|[neutral]|
|26 October 2010 - Finnish environmental management company Lassila & Tikanoja...|negative|[neutral]|
|A tinyurl link takes users to a scamming site promising that users can earn t...|negative|[neutral]|
|ADP News - Feb 13 , 2009 - Finnish retailer Kesko Oyj HEL : KESBV said today ...|negative|[neutral]|
|ADPnews - Sep 28 , 2009 - Finnish silicon wafers maker Okmetic Oyj HEL : OKM1...|

In [125]:
# 9.2

preds_df = preds.select('Review','Tweet',"class.result").toPandas()

# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out

# 9.3
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [114]:
# 9.4 We are going to use sklearn to evalute 
#      the results on test dataset:

from sklearn.metrics import classification_report

# 9.5
print (classification_report(preds_df['result'], preds_df['Review']))

              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         0
     neutral       0.92      0.71      0.80       734
    positive       0.46      0.57      0.51       224

    accuracy                           0.68       958
   macro avg       0.46      0.43      0.44       958
weighted avg       0.81      0.68      0.73       958



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
